In [ ]:
#!pip install delta-spark==1.2.1
#!pyspark --version

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master('local[*]')
    .appName('Quickstart DeltaLake Eng Academy')
    .config('spark.jars.packages', 'io.delta:delta-core_2.12:1.2.1')
    .config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
    .config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')
    .getOrCreate()
)

22/07/11 11:21:50 WARN Utils: Your hostname, Carloss-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.15.5 instead (on interface en0)
22/07/11 11:21:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/carlosbarbosa/.asdf/installs/python/3.9.13/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/carlosbarbosa/.ivy2/cache
The jars for the packages stored in: /Users/carlosbarbosa/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bac304d1-5cf3-4981-8633-e022858f9976;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.2.1 in central
	found io.delta#delta-storage;1.2.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 326ms :: artifacts dl 28ms
	:: modules in use:
	io.delta#delta-core_2.12;1.2.1 from central in [default]
	io.delta#delta-storage;1.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| s

In [88]:
sc = spark.sparkContext
sc.setLogLevel('ERROR')

In [69]:
df = spark.read.format('json').load('../data/')

In [5]:
columns = ['user_id', 'email']
df = df.select(columns)
df.columns

['user_id', 'email']

In [6]:
df.show(n=1, vertical=True, truncate=False)

-RECORD 0-------------------------
 user_id | 1703                   
 email   | daron.bailey@email.com 
only showing top 1 row



In [7]:
if df.rdd.getNumPartitions() > 3:
    df.coalesce(1).write.format('delta').mode('overwrite').save('../delta/bronze/user')
else:
    df.write.format('delta').mode('overwrite').save('../delta/bronze/user')

In [103]:
from delta.tables import DeltaTable

DeltaTable.forPath(spark, '../delta/bronze/user').history(1).show(vertical=True, truncate=False)

_ = DeltaTable.forPath(spark, '../delta/bronze/user').toDF()

-RECORD 0-----------------------------------------------------------------------------
 version             | 12                                                             
 timestamp           | 2022-07-05 18:05:33.507                                        
 userId              | null                                                           
 userName            | null                                                           
 operation           | WRITE                                                          
 operationParameters | {mode -> Overwrite, partitionBy -> []}                         
 job                 | null                                                           
 notebook            | null                                                           
 clusterId           | null                                                           
 readVersion         | 11                                                             
 isolationLevel      | Serializable        

In [9]:
ids = '1703, 3650'

In [10]:
_.where(f'user_id in ({ids})').show()

+-------+--------------------+
|user_id|               email|
+-------+--------------------+
|   1703|daron.bailey@emai...|
|   3650|jonah.barrows@ema...|
+-------+--------------------+



In [11]:
delta_object = DeltaTable.forPath(spark, '../delta/bronze/user')
delta_object

In [12]:
# before delete
_.where('user_id == 1703').show(truncate=False)
_.count()

+-------+----------------------+
|user_id|email                 |
+-------+----------------------+
|1703   |daron.bailey@email.com|
+-------+----------------------+



600

In [20]:
df.select('user_id').distinct().show()

+-------+
|user_id|
+-------+
|   5385|
|    720|
|   8887|
|   9952|
|   7032|
|   4161|
|   2364|
|   5099|
|   2961|
|   6762|
|   3909|
|    343|
|   9000|
|   9045|
|   8222|
|    402|
|   1312|
|   4746|
|   3650|
|   1805|
+-------+
only showing top 20 rows



In [14]:
ids_delete = '5385, 5099, 9000'


In [15]:
# delete
delta_object.delete(condition=f'user_id in ({ids_delete})')

In [16]:
_.count()

596

In [17]:
_.where('user_id == 1805').show()

+-------+--------------------+
|user_id|               email|
+-------+--------------------+
|   1805|renee.hahn@email.com|
+-------+--------------------+



In [18]:
# update
delta_object.update(
    condition='email = "renee.hahn@email.com"',
    set = { 'email': '"carlos.barbosa@owshq.com"'}
)

In [19]:
_.where('user_id == 1805').show(truncate=False)

+-------+------------------------+
|user_id|email                   |
+-------+------------------------+
|1805   |carlos.barbosa@owshq.com|
+-------+------------------------+



In [21]:
_.show()

+-------+--------------------+
|user_id|               email|
+-------+--------------------+
|   1703|daron.bailey@emai...|
|   3650|jonah.barrows@ema...|
|   8809|carla.hansen@emai...|
|   4606|tomas.ledner@emai...|
|      1|alyse.ortiz@email...|
|   9245|russell.kulas@ema...|
|   3425|armida.lehner@ema...|
|   4264|tad.sanford@email...|
|   1668|rosia.jones@email...|
|    343|candy.conroy@emai...|
|   7393|dulcie.gottlieb@e...|
|   3909|rodrigo.reynolds@...|
|   9952|jenna.bode@email.com|
|   2364|dan.herman@email.com|
|   1611|stanley.witting@e...|
|   1723|clarinda.kilback@...|
|   7032|charley.carroll@e...|
|    549|cameron.harris@em...|
|   4161|reyes.stracke@ema...|
|    503|jolynn.schulist@e...|
+-------+--------------------+
only showing top 20 rows



In [22]:
values = [(1703, 'luan.moreno@owshq.com'), (3650, 'mateus.oliveira@owshq.com')] 
cols   = _.columns

In [23]:
values, cols

([(1703, 'luan.moreno@owshq.com'), (3650, 'mateus.oliveira@owshq.com')],
 ['user_id', 'email'])

In [25]:
new_data = spark.createDataFrame(values, cols)
new_data.show(truncate=False)

+-------+-------------------------+
|user_id|email                    |
+-------+-------------------------+
|1703   |luan.moreno@owshq.com    |
|3650   |mateus.oliveira@owshq.com|
+-------+-------------------------+



In [37]:
# upserts
(
    delta_object.alias('old_data')
        .merge(
            new_data.alias('new_data'), condition = 'old_data.user_id = new_data.user_id'
        )
        .whenMatchedUpdateAll(
            condition = 'old_data.user_id = new_data.user_id'
        )
        .whenNotMatchedInsertAll()
    .execute()
)

In [35]:
ids_update = '1703, 3650'
_.where(f'user_id in ({ids_update})').show(truncate=False)

+-------+-------------------------+
|user_id|email                    |
+-------+-------------------------+
|1703   |luan.moreno@owshq.com    |
|3650   |mateus.oliveira@owshq.com|
+-------+-------------------------+



In [38]:
_.count()

596

In [44]:
delta_object.history().show(truncate=False, vertical=True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 8                                                                                                                                                                                                                                                                                  
 timestamp           | 2022-07-05 17:43:43.85                                                                                                                                                                                                                                                             
 userId              | null                                                                            

In [53]:
cols_history = ['version', 'timestamp', 'operation']
delta_object.history().select(cols_history).show(truncate=False)

+-------+-----------------------+---------+
|version|timestamp              |operation|
+-------+-----------------------+---------+
|8      |2022-07-05 17:43:43.85 |MERGE    |
|7      |2022-07-05 17:42:11.742|MERGE    |
|6      |2022-07-05 17:34:31.69 |UPDATE   |
|5      |2022-07-05 17:34:26.159|DELETE   |
|4      |2022-07-05 17:34:15.632|WRITE    |
|3      |2022-07-05 14:01:57.123|UPDATE   |
|2      |2022-07-05 13:59:26.742|DELETE   |
|1      |2022-07-05 13:57:54.893|DELETE   |
|0      |2022-07-05 13:37:57.416|WRITE    |
+-------+-----------------------+---------+



In [62]:
# time travel per version
time_travel_version_0 = (
    spark
        .read
        .format('delta')
        .option('versionAsOf', '0')
        .load('../delta/bronze/user/')
)

time_travel_version_0.show()
time_travel_version_0.count()

+-------+--------------------+
|user_id|               email|
+-------+--------------------+
|   1703|daron.bailey@emai...|
|   3650|jonah.barrows@ema...|
|   8809|carla.hansen@emai...|
|   4606|tomas.ledner@emai...|
|      1|alyse.ortiz@email...|
|   9245|russell.kulas@ema...|
|   3425|armida.lehner@ema...|
|   4264|tad.sanford@email...|
|   1668|rosia.jones@email...|
|    343|candy.conroy@emai...|
|   7393|dulcie.gottlieb@e...|
|   3909|rodrigo.reynolds@...|
|   9952|jenna.bode@email.com|
|   2364|dan.herman@email.com|
|   1611|stanley.witting@e...|
|   1723|clarinda.kilback@...|
|   7032|charley.carroll@e...|
|    549|cameron.harris@em...|
|   4161|reyes.stracke@ema...|
|    503|jolynn.schulist@e...|
+-------+--------------------+
only showing top 20 rows



600

In [61]:
# time travel per timestamp
time_travel_timestamp = (
    spark
        .read
        .format('delta')
        .option('timestampAsOf', '2022-07-05 17:43:43.85')
        .load('../delta/bronze/user/')
)

time_travel_timestamp.where(f'user_id in ({ids_update})').show(truncate=False)
time_travel_timestamp.count()

+-------+-------------------------+
|user_id|email                    |
+-------+-------------------------+
|1703   |luan.moreno@owshq.com    |
|3650   |mateus.oliveira@owshq.com|
+-------+-------------------------+



596

In [67]:
# verify changes beetween to versions delta
time_travel_version_0.exceptAll(time_travel_timestamp).show(truncate=False)

+-------+-------------------------+
|user_id|email                    |
+-------+-------------------------+
|9000   |josefa.herman@email.com  |
|3650   |jonah.barrows@email.com  |
|9000   |herschel.turner@email.com|
|5099   |chang.senger@email.com   |
|1703   |daron.bailey@email.com   |
|5385   |winston.kirlin@email.com |
|1805   |renee.hahn@email.com     |
+-------+-------------------------+



In [71]:
# evolution schema
columns = ['user_id', 'email', 'gender']
df = df.select(columns)
df.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)



In [99]:
df.write.format('delta').mode('overwrite').option('mergeSchema', True).save('../delta/bronze/user') # mergeSchema / overwriteSchema

In [100]:
delta_object.history(1).show(vertical=True, truncate=False)

-RECORD 0-----------------------------------------------------------------------------
 version             | 12                                                             
 timestamp           | 2022-07-05 18:05:33.507                                        
 userId              | null                                                           
 userName            | null                                                           
 operation           | WRITE                                                          
 operationParameters | {mode -> Overwrite, partitionBy -> []}                         
 job                 | null                                                           
 notebook            | null                                                           
 clusterId           | null                                                           
 readVersion         | 11                                                             
 isolationLevel      | Serializable        

In [104]:
_.show()

+-------+--------------------+-----------+
|user_id|               email|     gender|
+-------+--------------------+-----------+
|   3395|marcos.collier@em...|   Bigender|
|   1556|elina.hills@email...| Non-binary|
|   1879|enedina.schroeder...|     Female|
|   7805|colin.ryan@email.com|       Male|
|   3982|dallas.boyle@emai...| Polygender|
|   7274|grover.towne@emai...|    Agender|
|   3184|dexter.schmitt@em...|Genderfluid|
|    550|novella.weber@ema...|Genderqueer|
|   8365|lesley.mccullough...|       Male|
|   4942|marti.marks@email...|   Bigender|
|   8327|shawnna.keebler@e...|Genderfluid|
|   9464|guillermo.beahan@...|   Bigender|
|   4123|sid.bechtelar@ema...| Non-binary|
|   2281|merrill.upton@ema...|     Female|
|   6998|felipe.ward@email...|    Agender|
|   7440|willie.walsh@emai...|       Male|
|   8397|jae.krajcik@email...| Polygender|
|   9437|wilfredo.bailey@e...| Polygender|
|   3148|josefa.marvin@ema...|     Female|
|   2535|loyd.hintz@email.com| Polygender|
+-------+--

In [107]:
delta_object.generate('symlink_format_manifest')

In [106]:
df.coalesce(1).write.format('delta').mode('overwrite').save('../delta/bronze/user')

In [108]:
df.coalesce(1).write.format('parquet').mode('overwrite').save('../parquet/user')

In [124]:
# migrate data lake parquet to delta
delta_object.convertToDelta(spark, 'parquet.`../parquet/user/`')

The table you are trying to convert is already a delta table


In [127]:
delta_object.history(1).show(truncate=False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------
 version             | 13                                                             
 timestamp           | 2022-07-05 18:08:17.527                                        
 userId              | null                                                           
 userName            | null                                                           
 operation           | WRITE                                                          
 operationParameters | {mode -> Overwrite, partitionBy -> []}                         
 job                 | null                                                           
 notebook            | null                                                           
 clusterId           | null                                                           
 readVersion         | 12                                                             
 isolationLevel      | Serializable        